In [0]:
One of the SQL questions recently asked in WinWire interview.
Given us happiness_tbl, we need to display a few specific Rows always at top of the query results.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, length, when

# Create a Spark session
spark = SparkSession.builder.appName("Happiness Table").getOrCreate()

# Define the schema for happiness_tbl
schema = StructType([
    StructField("ranking", IntegerType(), True),
    StructField("country", StringType(), True)
])

# Data for happiness_tbl
data_happiness = [
    (1, 'Finland'),
    (2, 'Denmark'),
    (3, 'Iceland'),
    (4, 'Israel'),
    (5, 'Netherlands'),
    (6, 'Sweden'),
    (7, 'Norway'),
    (8, 'Switzerland'),
    (9, 'Luxembourg'),
    (128, 'Srilanka'),
    (126, 'India')
]

# Create a DataFrame for happiness_tbl
df_happiness = spark.createDataFrame(data_happiness, schema)

# Show the DataFrame
df_happiness.display()


ranking,country
1,Finland
2,Denmark
3,Iceland
4,Israel
5,Netherlands
6,Sweden
7,Norway
8,Switzerland
9,Luxembourg
128,Srilanka


In [0]:
df_happiness.createOrReplaceTempView('happiness_tbl')

In [0]:
%sql
select *,
LEN(ranking) length
from happiness_tbl
order by (case when LEN(ranking)>1 then 0 else 1 end), 
	     ranking

ranking,country,length
126,India,3
128,Srilanka,3
1,Finland,1
2,Denmark,1
3,Iceland,1
4,Israel,1
5,Netherlands,1
6,Sweden,1
7,Norway,1
8,Switzerland,1


In [0]:
# Calculate the length of the 'ranking' column
df_with_length = df_happiness.withColumn("length", length(col("ranking").cast("string")))

# Conditional ordering: First order by whether the length of 'ranking' > 1, then by 'ranking'
df_final = df_with_length.orderBy(
    when(col("length") > 1, 0).otherwise(1).asc(),
    col("ranking").asc()
)

# Show the final result
df_final.display()

ranking,country,length
126,India,3
128,Srilanka,3
1,Finland,1
2,Denmark,1
3,Iceland,1
4,Israel,1
5,Netherlands,1
6,Sweden,1
7,Norway,1
8,Switzerland,1
